In [1]:
import pandas as pd
import numpy as np
import json
np.random.seed(0)
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from keras.preprocessing import text, sequence
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


In [2]:
def parse(file):
    for l in open(file, 'r'):
        yield json.loads(l)

In [3]:
data = parse('news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json')

In [4]:
df = pd.DataFrame(data)

In [5]:
df = df.drop(df.columns[0], axis=1)

In [6]:
target = df.is_sarcastic
df = df.headline

In [8]:
y = pd.get_dummies(target).values

In [9]:
y

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [19]:
tokenizer = text.Tokenizer(num_words=70000)
tokenizer.fit_on_texts(list(df))
list_tokenized_train = tokenizer.texts_to_sequences(df)
X = sequence.pad_sequences(list_tokenized_train, maxlen=100)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25)

In [22]:
clf = xgb.XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 95.74%
Validation accuracy: 73.43%
